In [1]:
states = ['cr7', 'cr28']
obs = ['Greatest', 'of', 'all', 'time']
seq = [2,1,0] 
ini = [1,0] 

'''trans = [[float(input(f"Enter the s{i+1} to s{j+1}")) for j in range(len(states))]for i in range(len(states))]

emi = [
    [0.1, 0.6, 0.2, 0.1], 
    [0.5, 0.3, 0.1, 0.1]
]'''


emi=[[0.6,0.1,0.3],[0.1,0.7,0.2]]
trans=[[0.7,0.3],[0.5,0.5]]

def forward(states, obs, seq, ini, trans, emi):
    num_states = len(states)
    print(num_states)
    T = len(seq)
    alpha = [[0] * T for _ in range(num_states)]
    print(alpha)
    for i in range(num_states):
        alpha[i][0] = ini[i] * emi[i][seq[0]]
    print(alpha)
    for t in range(1, T):
        for j in range(num_states):
            sum_alpha = 0
            for i in range(num_states):
                sum_alpha += alpha[i][t - 1] * trans[i][j]
            alpha[j][t] = sum_alpha * emi[j][seq[t]]

    print("Alpha matrix:")
    for row in alpha:
        print(row)

    prob = sum(alpha[i][T - 1] for i in range(num_states))
    print(f"\nTotal Probability of the sequence: {prob}")

    return alpha
alpha_values = forward(states, obs, seq, ini, trans, emi)


2
[[0, 0, 0], [0, 0, 0]]
[[0.3, 0, 0], [0.0, 0, 0]]
Alpha matrix:
[0.3, 0.021, 0.027719999999999998]
[0.0, 0.063, 0.0037800000000000004]

Total Probability of the sequence: 0.0315


In [2]:
states = ['cr7', 'cr28']
obs = ['Greatest', 'of', 'all', 'time']
seq = [1, 3, 0]  # 'of', 'time', 'Greatest'
ini = [0.5, 0.5] 

trans = [
    [0.6, 0.4], 
    [0.5, 0.5]
]

emi = [
    [0.1, 0.6, 0.2, 0.1], 
    [0.5, 0.3, 0.1, 0.1]
]
def backward(states, obs, seq, ini, trans, emi):
    num_states = len(states)
    T = len(seq)+1
    beta = [[0] * (T) for _ in range(num_states)]

    for i in range(num_states):
        beta[i][T-1] = 0.5

    for t in range(T - 2, -1, -1):
        for i in range(num_states):
            beta[i][t] = sum(
                trans[i][j] * emi[i][seq[t]] * beta[j][t+1]
                for j in range(num_states)
            )
            
    print("\nBeta matrix:")
    for row in beta:
        print(row)
    prob=sum ([beta[i][0] for i in range(num_states)])
    '''prob = sum(
        ini[i] * emi[i][seq[0]] * beta[i][0]
        for i in range(num_states)
    )'''
    print(f"\nTotal Probability of the sequence (Backward): {prob}")

    return beta
beta_values = backward(states, obs, seq, ini, trans, emi)


Beta matrix:
[0.008280000000000001, 0.013000000000000001, 0.05, 0.5]
[0.004200000000000001, 0.015000000000000001, 0.25, 0.5]

Total Probability of the sequence (Backward): 0.012480000000000002


In [6]:
import nltk
from nltk import PCFG,ViterbiParser,InsideChartParser
import numpy as np
import pandas as pd
grammar =  PCFG.fromstring("""
S -> NP VP [1.0]
VP -> V NP [0.7] | VP PP [0.3]
NP -> Det N [0.5] | NP PP [0.2] | 'John' [0.3]
PP -> P NP [1.0]
V -> 'saw' [0.2] | 'ate' [0.8]
Det -> 'the' [0.6] | 'a' [0.4]
N -> 'man' [0.5] | 'telescope' [0.5]
P -> 'with' [0.4] | 'in' [0.6]
""")
parser=ViterbiParser(grammar)
print(parser)
sentence = "John saw a man with a telescope"
tokens = sentence.split()
#print(parser.parse(tokens))

for j in parser.parse(tokens):
    print(j.pretty_print())


<ViterbiParser for <Grammar with 15 productions>>
      S                                    
  ____|___________                          
 |                VP                       
 |         _______|________                 
 |        VP               PP              
 |     ___|___         ____|___             
 |    |       NP      |        NP          
 |    |    ___|___    |     ___|______      
 NP   V  Det      N   P   Det         N    
 |    |   |       |   |    |          |     
John saw  a      man with  a      telescope

None


In [24]:
# CYK Used defined
import numpy as pd
import nltk
# Define a Probabilistic Context-Free Grammar (PCFG)
pcfg = nltk.PCFG.fromstring("""
 S -> NP VP [0.9]
 S -> VP [0.1]
 VP -> V NP [0.5]
 VP -> V [0.5]
 NP -> Det N [0.3]
 NP -> N [0.7]
 N -> 'cat' [0.2]
 N -> 'book' [0.2]
 N -> 'bird' [0.2]
 N -> 'dog' [0.4]
 V -> 'read' [0.1]
 V -> 'chased' [0.6]
 V -> 'ate' [0.3]
 Det -> 'the' [0.5]
 Det -> 'a' [0.5]
""")

sentences=[
 "the cat chased the dog",
 "the dog chased the cat",
 "the dog chased the bird",
 "Book read"
]


# Input sentence to parse

#sentence = "the cat chased the dog".split()

def cyk_parse_with_probability(pcfg, sentence):
  n = len(sentence)
  table = [[[] for _ in range(n)] for _ in range(n)]
  # Initialize the table with production probabilities
  for i in range(n):
    for prod in pcfg.productions(rhs=sentence[i]):
      table[i][i] = [(prod.lhs(), prod.prob())]
  for length in range(2, n + 1):
    for i in range(n - length + 1):
      j = i + length - 1
      for k in range(i, j):
        for prod in pcfg.productions():
          for left, left_prob in table[i][k]:
            for right, right_prob in table[k + 1][j]:
              if prod.rhs() == (left, right):
                prob = left_prob * right_prob *prod.prob()
                if table[i][j] is None:
                  table[i][j] = [(prod.lhs(), prob)]
                else:
                  table[i][j].append((prod.lhs(),prob))
  print(table)
  # Check if the start symbol is in the top-right cell
  if pcfg.start() in [lhs for lhs, prob in table[0][n - 1]]:
    # Find the probability for the full sentence
    for lhs, prob in table[0][n - 1]:
      if lhs == pcfg.start():
        return prob
  return 0.0


# Parse the sentence and calculate the probability
for sent in sentences:
  probability = cyk_parse_with_probability(pcfg,sent.lower().split())
  if probability > 0:
    print(f'The sentence "{sent}" is grammatically correct with a probability of:', probability)
  else:
    print(f"The sentence \"{sent}\" is not grammatically correct.")

[[[(Det, 0.5)], [(NP, 0.03)], [], [], [(S, 0.00048599999999999994)]], [[], [(N, 0.2)], [], [], []], [[], [], [(V, 0.6)], [], [(VP, 0.018)]], [[], [], [], [(Det, 0.5)], [(NP, 0.06)]], [[], [], [], [], [(N, 0.4)]]]
The sentence "the cat chased the dog" is grammatically correct with a probability of: 0.00048599999999999994
[[[(Det, 0.5)], [(NP, 0.06)], [], [], [(S, 0.00048599999999999994)]], [[], [(N, 0.4)], [], [], []], [[], [], [(V, 0.6)], [], [(VP, 0.009)]], [[], [], [], [(Det, 0.5)], [(NP, 0.03)]], [[], [], [], [], [(N, 0.2)]]]
The sentence "the dog chased the cat" is grammatically correct with a probability of: 0.00048599999999999994
[[[(Det, 0.5)], [(NP, 0.06)], [], [], [(S, 0.00048599999999999994)]], [[], [(N, 0.4)], [], [], []], [[], [], [(V, 0.6)], [], [(VP, 0.009)]], [[], [], [], [(Det, 0.5)], [(NP, 0.03)]], [[], [], [], [], [(N, 0.2)]]]
The sentence "the dog chased the bird" is grammatically correct with a probability of: 0.00048599999999999994
[[[(N, 0.2)], []], [[], [(V, 0.1)

In [21]:
import nltk

# Define a Probabilistic Context-Free Grammar (PCFG)
pcfg = nltk.PCFG.fromstring("""
 S -> NP VP [0.9]
 S -> VP [0.1]
 VP -> V NP [0.5]
 VP -> V [0.5]
 NP -> Det N [0.3]
 NP -> N [0.7]
 N -> 'cat' [0.2]
 N -> 'book' [0.2]
 N -> 'bird' [0.2]
 N -> 'dog' [0.4]
 V -> 'read' [0.1]
 V -> 'chased' [0.6]
 V -> 'ate' [0.3]
 Det -> 'the' [0.5]
 Det -> 'a' [0.5]
""")

sentences = [
    "the cat chased the dog",
    "the dog chased the cat",
    "the dog chased the bird",
    "Book read"
]

def cyk_parse_with_probability(pcfg, sentence):
    n = len(sentence)
    table = [[[] for _ in range(n)] for _ in range(n)]

    for i in range(n):
        for prod in pcfg.productions(rhs=sentence[i]):
            table[i][i].append((prod.lhs(), prod.prob()))

    for length in range(2, n + 1):
        for i in range(n - length + 1):
            j = i + length - 1
            for k in range(i, j):
                for prod in pcfg.productions():
                    if len(prod.rhs()) == 2:
                        B, C = prod.rhs()
                        for B_lhs, B_prob in table[i][k]:
                            for C_lhs, C_prob in table[k + 1][j]:
                                if B_lhs == B and C_lhs == C:
                                    prob = B_prob * C_prob * prod.prob()
                                    table[i][j].append((prod.lhs(), prob))

    # Return the max probability of S in the top cell
    return max((prob for lhs, prob in table[0][n - 1] if lhs == pcfg.start()), default=0.0)

# Evaluate each sentence
for sent in sentences:
    prob = cyk_parse_with_probability(pcfg, sent.lower().split())
    if prob > 0:
        print(f'"{sent}" is grammatically correct with probability {prob}')
    else:
        print(f'"{sent}" is not grammatically correct.')


"the cat chased the dog" is grammatically correct with probability 0.00048599999999999994
"the dog chased the cat" is grammatically correct with probability 0.00048599999999999994
"the dog chased the bird" is grammatically correct with probability 0.00048599999999999994
"Book read" is not grammatically correct.


In [22]:
import nltk

pcfg=PCFG.fromstring('''
S -> NP VP [0.9]
S -> VP [0.1]
VP -> V NP [0.5]
VP -> V [0.5]
NP -> Det N [0.3]
NP -> N [0.7]
N -> 'cat' [0.2]
N -> 'book' [0.2]
N -> 'bird' [0.2]
N -> 'dog' [0.4]
V -> 'read' [0.1]
V -> 'chased' [0.6]
V -> 'ate' [0.3]
Det -> 'the' [0.5]
Det -> 'a' [0.5]
''')
sentences = [
    "the cat chased the dog",
    "the dog chased the cat",
    "the dog chased the bird",
    "Book read"
]

for sent in sentences:
    probability = cyk_parse_with_probability(pcfg,sent.lower().split())
    if probability > 0:
        print(f'The sentence "{sent}" is grammatically correct with a probability of:', probability)
    else:
        print(f"The sentence \"{sent}\" is not grammatically correct.")

The sentence "the cat chased the dog" is grammatically correct with a probability of: 0.00048599999999999994
The sentence "the dog chased the cat" is grammatically correct with a probability of: 0.00048599999999999994
The sentence "the dog chased the bird" is grammatically correct with a probability of: 0.00048599999999999994
The sentence "Book read" is not grammatically correct.


In [8]:
import numpy as np

seq= [1,3,0]  

inp = np.array([0.5, 0.5]) 
trans = np.array([
    [0.6, 0.4],  
    [0.5, 0.5]  
])
emi= np.array([
    [0.1, 0.6, 0.2, 0.1],  
    [0.5, 0.3, 0.1, 0.1] 
])

def forward(seq,trans,emi,inp):
    t=len(seq)
    alpha=[[0]*(t) for i in range(len(inp))]
    for i in range(len(inp)):
        alpha[i][0]=inp[i]
    for i in range(0,t-1):
        for j in range(len(inp)):
            maxi=0
            for k in range(len(inp)):
                s=alpha[k][i]*trans[k][j]
                maxi=max(maxi,s)
            alpha[j][i+1]=maxi*emi[j][seq[i]]
    q=""
    t=len(seq)
    
    for i in range(len(seq)-1,0,-1):
        max1=alpha[0][i]
        maxi1=alpha[1][i]
        real=max(max1,maxi1)
        if(real==max1):
            q+=str(1)+" "
        if(real==maxi1):
            q+=str(2)+" "
        print(q)
    print("Sequence:",q,"S")   
    print(alpha)
forward(seq,trans,emi,inp)

1 
1 1 
Sequence: 1 1  S
[[0.5, 0.18, 0.0108], [0.5, 0.075, 0.0072]]
